In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/scaling-and-smote/train_smote.pkl
/kaggle/input/scaling-and-smote/train_test_split.pkl


In [2]:
import joblib

X_train_smote, y_train_smote = joblib.load('/kaggle/input/scaling-and-smote/train_smote.pkl')
X_test = joblib.load('/kaggle/input/scaling-and-smote/train_test_split.pkl')[1]

In [3]:
##### import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam

# Reshape the data to 3D for LSTM (samples, timesteps, features)
X_train_lstm = X_train_smote.values.reshape((X_train_smote.shape[0], X_train_smote.shape[1], 1))
X_test_lstm = X_test.values.reshape((X_test.shape[0], X_test.shape[1], 1))

# Define LSTM model
lstm_model = Sequential()
lstm_model.add(LSTM(64, input_shape=(X_train_lstm.shape[1], 1)))
lstm_model.add(Dense(64, activation='relu'))
lstm_model.add(Dense(1, activation='sigmoid'))

optimizer = Adam(learning_rate=0.001, clipnorm=1.0)
lstm_model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
lstm_model.fit(X_train_lstm, y_train_smote, epochs=10, batch_size=32)

# Extract features using LSTM
lstm_features = lstm_model.predict(X_train_lstm)
lstm_features_test = lstm_model.predict(X_test_lstm)

2025-04-20 18:37:22.486132: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745174242.668095      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745174242.720932      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
I0000 00:00:1745174255.440900      31 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(sha

Epoch 1/10


I0000 00:00:1745174262.452148      94 cuda_dnn.cc:529] Loaded cuDNN version 90300


94934/94934 ━━━━━━━━━━━━━━━━━━━━ 565s 6ms/step - accuracy: 0.7421 - loss: 0.4634
Epoch 2/10
94934/94934 ━━━━━━━━━━━━━━━━━━━━ 555s 6ms/step - accuracy: 0.8478 - loss: 0.3513
Epoch 3/10
94934/94934 ━━━━━━━━━━━━━━━━━━━━ 549s 6ms/step - accuracy: 0.8774 - loss: 0.3151
Epoch 4/10
94934/94934 ━━━━━━━━━━━━━━━━━━━━ 549s 6ms/step - accuracy: 0.8764 - loss: 0.3076
Epoch 5/10
94934/94934 ━━━━━━━━━━━━━━━━━━━━ 546s 6ms/step - accuracy: 0.9470 - loss: 0.1648
Epoch 6/10
94934/94934 ━━━━━━━━━━━━━━━━━━━━ 549s 6ms/step - accuracy: 0.9485 - loss: 0.1568
Epoch 7/10
94934/94934 ━━━━━━━━━━━━━━━━━━━━ 547s 6ms/step - accuracy: 0.9489 - loss: 0.1559
Epoch 8/10
94934/94934 ━━━━━━━━━━━━━━━━━━━━ 548s 6ms/step - accuracy: 0.9455 - loss: 0.1657
Epoch 9/10
94934/94934 ━━━━━━━━━━━━━━━━━━━━ 546s 6ms/step - accuracy: 0.9497 - loss: 0.1511
Epoch 10/10
94934/94934 ━━━━━━━━━━━━━━━━━━━━ 547s 6ms/step - accuracy: 0.9502 - loss: 0.1496
94934/94934 ━━━━━━━━━━━━━━━━━━━━ 175s 2ms/step
13524/13524 ━━━━━━━━━━━━━━━━━━━━ 25s 2ms/st

In [4]:
np.save('lstm_features_train.npy', lstm_features)
np.save('lstm_features_test.npy', lstm_features_test)
features = {'train': lstm_features, 'test': lstm_features_test}
np.save('lstm_features.npy', features, allow_pickle=True)
lstm_model.save('lstm_model.h5')

In [5]:
from IPython.display import FileLink
FileLink(r'lstm_features_train.npy')

/kaggle/working/lstm_features_train.npy

In [6]:
from IPython.display import FileLink
FileLink(r'lstm_features_test.npy')

/kaggle/working/lstm_features_test.npy

In [7]:
from IPython.display import FileLink
FileLink(r'lstm_features.npy')

/kaggle/working/lstm_features.npy

In [8]:
from IPython.display import FileLink
FileLink(r'lstm_model.h5')

/kaggle/working/lstm_model.h5